# Import Libraries

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Obtain Economic Indicator Data

Create BigQuery client

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= '../Other/Keys/olympics-330304-2824eb690cdc.json'
client = bigquery.Client()

Get world bank wdi dataset

In [3]:
wb_dataset_id = client.dataset("world_bank_wdi", project="bigquery-public-data")
wb_dataset = client.get_dataset(wb_dataset_id)

View tables in world bank dataset

In [4]:
wb_tables = list(client.list_tables(wb_dataset))
for table in wb_tables:
    print(table.table_id)

country_series_definitions
country_summary
footnotes
indicators_data
series_summary
series_time


View world bank indicators_data table

In [5]:
indicator_table_id = wb_dataset_id.table("indicators_data")
indicator_table = client.get_table(indicator_table_id)
[print(item) for item in indicator_table.schema]
client.list_rows(indicator_table, max_results=5).to_dataframe()

SchemaField('country_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('country_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('value', 'FLOAT', 'NULLABLE', '', (), None)
SchemaField('year', 'INTEGER', 'NULLABLE', '', (), None)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1860: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,country_name,country_code,indicator_name,indicator_code,value,year
0,Kuwait,KWT,Commercial service imports (current US$),TM.VAL.SERV.CD.WT,1.432269e+10,2010
1,Kuwait,KWT,"Communications, computer, etc. (% of service e...",BX.GSR.CMCP.ZS,1.595238e+01,1994
2,Kuwait,KWT,"Communications, computer, etc. (% of service i...",BM.GSR.CMCP.ZS,2.227488e+01,1983
3,Kuwait,KWT,"Computer, communications and other services (%...",TX.VAL.OTHR.ZS.WT,-4.121148e-13,1994
4,Kuwait,KWT,"Contraceptive prevalence, any methods (% of wo...",SP.DYN.CONU.ZS,3.460000e+01,1987


Query country indicators from indicators_data table

In [6]:
indicators_query = """
        SELECT *
        FROM `bigquery-public-data.world_bank_wdi.indicators_data`
        WHERE indicator_name IN ('Population, total',
                                 'GDP per capita, PPP (constant 2017 international $)',
                                 'Urban population (% of total population)',
                                 'Gini index (World Bank estimate)',
                                 'Current health expenditure (% of GDP)',
                                 'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
                                 'International migrant stock (% of population)',
                                 'Government expenditure on education, total (% of GDP)',
                                 'Population ages 20-24, female (% of female population)',
                                 'Population ages 20-24, male (% of male population)',
                                 'Population ages 25-29, female (% of female population)',
                                 'Population ages 25-29, male (% of male population)',
                                 'Population ages 30-34, female (% of female population)',
                                 'Population ages 30-34, male (% of male population)')
        AND YEAR >= 2010"""
indicators_query_job = client.query(indicators_query)
indicators_df = indicators_query_job.to_dataframe()
indicators_df.head()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Kyrgyz Republic,KGZ,Gini index (World Bank estimate),SI.POV.GINI,27.4,2012
1,Lithuania,LTU,Gini index (World Bank estimate),SI.POV.GINI,37.4,2015
2,Luxembourg,LUX,Gini index (World Bank estimate),SI.POV.GINI,31.7,2016
3,Madagascar,MDG,Gini index (World Bank estimate),SI.POV.GINI,42.6,2012
4,Hungary,HUN,Gini index (World Bank estimate),SI.POV.GINI,30.8,2012


Drop unneeded columns from indicators_df

In [8]:
indicators_df.drop(columns=['country_code', 'indicator_code'], inplace=True)
indicators_df.head()

,country_name,indicator_name,value,year
0,Kyrgyz Republic,Gini index (World Bank estimate),27.4,2012
1,Lithuania,Gini index (World Bank estimate),37.4,2015
2,Luxembourg,Gini index (World Bank estimate),31.7,2016
3,Madagascar,Gini index (World Bank estimate),42.6,2012
4,Hungary,Gini index (World Bank estimate),30.8,2012


Use max_year function to determine which year is most recently available datapoint for each feature by country

In [10]:
def max_year(row):
    max_year = max(indicators_df[(indicators_df['country_name'] == row['country_name']) & 
                                 (indicators_df['indicator_name'] == row['indicator_name'])]['year'].values)    
    if row['year'] == max_year:  
        return 1
    else:
        return 0
indicators_df['max_year'] = indicators_df.apply(max_year, axis=1)
indicators_df.head()

,country_name,indicator_name,value,year,max_year
0,Kyrgyz Republic,Gini index (World Bank estimate),27.4,2012,0
1,Lithuania,Gini index (World Bank estimate),37.4,2015,0
2,Luxembourg,Gini index (World Bank estimate),31.7,2016,0
3,Madagascar,Gini index (World Bank estimate),42.6,2012,1
4,Hungary,Gini index (World Bank estimate),30.8,2012,0


Filter indicators_df to only include most recently available year's data for each feature

In [19]:
max_indicators_df = indicators_df[indicators_df['max_year']==1]
max_indicators_df.drop(columns=['year', 'max_year'], inplace=True)
max_indicators_df.reset_index(drop=True, inplace=True)
max_indicators_df.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,country_name,indicator_name,value
0,Madagascar,Gini index (World Bank estimate),42.6
1,Czech Republic,Gini index (World Bank estimate),25.0
2,Lesotho,Gini index (World Bank estimate),44.9
3,Finland,Gini index (World Bank estimate),27.3
4,North Macedonia,Gini index (World Bank estimate),33.0


Pivot max_indicators_df to have country as row, indicator as column and value as datapoints

In [20]:
max_indicators_df = max_indicators_df.pivot(index='country_name', columns='indicator_name', values='value')
max_indicators_df.reset_index(inplace=True)
max_indicators_df.head()

indicator_name,country_name,Current health expenditure (% of GDP),"GDP per capita, PPP (constant 2017 international $)",Gini index (World Bank estimate),"Government expenditure on education, total (% of GDP)",International migrant stock (% of population),"PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)","Population ages 20-24, female (% of female population)","Population ages 20-24, male (% of male population)","Population ages 25-29, female (% of female population)","Population ages 25-29, male (% of male population)","Population ages 30-34, female (% of female population)","Population ages 30-34, male (% of male population)","Population, total",Urban population (% of total population)
0,Afghanistan,9.395727,1978.961579,NaN,3.21378,1.175547,56.910808,10.073862,10.098379,7.907260,8.063549,6.221683,6.503323,38928341.0,26.026000
1,Africa Eastern and Southern,5.925882,3371.814704,NaN,4.71930,1.734768,35.512988,9.218579,9.355607,7.906863,7.947182,6.763558,6.723442,677243299.0,36.783306
2,Africa Western and Central,4.004790,3995.039927,NaN,3.03285,2.079691,58.064482,8.940792,9.041989,7.522335,7.578698,6.425444,6.427822,458803476.0,47.848625
3,Albania,5.262714,13295.410885,33.2,3.94576,1.989036,18.200603,7.853874,8.013442,8.138641,8.828823,7.168070,8.372488,2837743.0,62.112000
4,Algeria,6.218427,10681.679297,27.6,6.10036,0.611072,38.884011,6.769581,6.910889,8.120237,8.179230,8.527349,8.474931,43851043.0,73.733000


Save max_indicators_df to csv file

In [21]:
max_indicators_df.to_csv('Data/indicators.csv', index=False)

# Scrape Medal Count Data

Get html with 2020 olympics national medal counts data

In [22]:
response_headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
url  = 'https://www.cbssports.com/olympics/news/tokyo-olympics-final-medal-table-usa-leads-in-in-gold-silver-bronze-and-overall-with-more-than-100-medals/'
response = requests.get(url, headers = response_headers, timeout=5)

Scrape medal count data from html code and save to medal_df

In [23]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'TableBase NOTSELECTED OlympicsTable'})
rows = table.find_all('tr')
data = []
for row in rows:
    row_data = []
    for datapoint in row.find_all('p'):
        row_data.append(datapoint.text.replace('\t', '').strip())
    data.append(row_data)
medal_df = pd.DataFrame(data)
medal_df.columns = medal_df.loc[0]
medal_df.drop(axis=0, index=0, inplace=True)
medal_df.head()

,COUNTRY,GOLD,SILVER,BRONZE,TOTALS
1,United States,39,41,33,113
2,China,38,32,18,88
3,Japan,27,14,17,58
4,Great Britain,22,21,22,65
5,Russian Olympic Committee,20,28,23,71


Save medal_df to csv file

In [24]:
medal_df.to_csv('Data/medals.csv', index=False)

# Scrape list of olympic nations

Get url with list of olympic nations using requests.get

In [4]:
nations_response_headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
nations_url  = 'https://olympics.com/ioc/national-olympic-committees'
nations_response = requests.get(nations_url, headers = nations_response_headers, timeout=5)

Scrape list of olympic nations from html and save to olympic_countries_df

In [35]:
nations_soup = BeautifulSoup(nations_response.content, 'html.parser')
nations_table = nations_soup.find('div', {'class': 'row row-cols-2 row-cols-md-3 row-cols-lg-4'})
countries = nations_table.find_all('a')
countries_list = []
for country in countries:
    countries_list.append(country.find('h6').text)
olympic_countries_df = pd.DataFrame(countries_list, columns=['Country'])
print(olympic_countries_df.shape)
olympic_countries_df.head()

(206, 1)


,Country
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


Save olympic_countries_df to csv file

In [36]:
olympic_countries_df.to_csv('Data/olympic_countries.csv', index=False)